In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WRITING_DPO_CONFIG, SMOL_LM_135M

cfg = WRITING_DPO_CONFIG
cfg.train_batch_size = 1
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-05 11:03:35,183	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-05 11:03:38.649 | INFO     | trl_wrapper.trainer_wrapper:__init__:306 - Setting padding side to: right
2025-04-05 11:03:38.662 | INFO     | trl_wrapper.trainer_wrapper:init_model:317 - Using device: mps
2025-04-05 11:03:38.663 | INFO     | trl_wrapper.trainer_wrapper:init_model:331 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [3]:
wrapper.init_data_module()

2025-04-05 11:03:43.127 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:357 - Using chat template override: smollmv2
2025-04-05 11:03:43.128 | INFO     | trl_wrapper.wrapper_config:__init__:136 - Cache dir: ../dataset_caches/writing_d_p_o_data_module


In [4]:
wrapper.data_module.train_dataset

Dataset({
    features: ['instruction_id', 'chosen', 'rejected', 'score_chosen', 'score_rejected', 'model_chosen', 'model_rejected', 'prompt'],
    num_rows: 9299
})

In [5]:
wrapper.init_trainer()

2025-04-05 11:03:49.051 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:394 - Saving output to: ./runs/04-05-11-3-770754-smollm2-135m-instruct--txt-bt-dpo
2025-04-05 11:03:49.052 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:417 - Initializing trainer, run_name: 04-05-11-3-770754-smollm2-135m-instruct--txt-bt-dpo, wandb project: gutenberg
2025-04-05 11:03:49.052 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:424 - logprobs cache location: ../dataset_caches/writing_d_p_o_data_module/b68b303b/ref_logprobs_cache peft config: False
Tokenizing eval dataset: 100%|██████████| 1034/1034 [00:01<00:00, 601.00 examples/s]


In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [6]:
wrapper.train()

2025-04-05 11:04:11.153 | INFO     | trl_wrapper.trainer_wrapper:train:666 - Starting training.
/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
2025-04-05 11:04:11.356 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:137 - Generating samples...
2025-04-05 11:04:11.356 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:59 - Generating policy samples, max length: 1024...


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
2025-04-05 11:04:31.470 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:86 - Generating reference samples, max length: 1024...
2025-04-05 11:04:36.088 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:141 - Generated samples.
2025-04-05 11:04:36.102 | INFO     | synthetic_data.utils:log_to_file:331 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ policy                                             │ ref                                                │ chosen                                             │ rejected                                           │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ Write a paragraph in a fast-paced, action-oriented │ Incorporate descriptive passages about the         │ Use a mix of action, dialogue, and description to  │ With a grunt of effort, Kaelan stagger

Step,Training Loss,Validation Loss


KeyboardInterrupt: 